In [36]:
import sys
sys.path.append("/home/benjamin/RewardCurriculum/")

import os
os.system("export MKL_SERVICE_FORCE_INTEL=1")

import gymnasium as gym
import panda_gym
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
import json
import numpy as np
from collections import defaultdict
import tqdm

import os
from copy import deepcopy
from utils.configs import get_config
from utils.env_wrappers import make_vec_env, get_env
from argparse import Namespace
import cv2
from IPython.display import HTML

In [4]:
os.chdir("/home/benjamin/RewardCurriculum")
folder_path = "/home/benjamin/RewardCurriculum/results/panda_stack_long/2024-10-17_00-06-08_PandaMultiRewardStackDense-v3_tqc_settersolver_2"

args = Namespace()
args.env_name = json.load(open(os.path.join(folder_path, "config.json"), "r"))["environment"]["env_name"]
args.seed = json.load(open(os.path.join(folder_path, "config.json"), "r"))["seed"]
args.config_path = os.path.join(folder_path, "config_original.json")
args.continue_from = None

remaining_args = ["--environment.wrapper_kwargs.0.reward_threshold", "-0.05"]
# remaining_args.__setattr__[] = "-0.05"
config = get_config(args.config_path, args, remaining_args)
config["environment"]["wrappers"] += ["SingleTaskRewardWrapper"]
config["environment"]["wrapper_kwargs"] += [{}]

In [6]:
make_env_fn = lambda wrappers, wrapper_kwargs, ignore_keyword="ignore" : get_env(config["environment"]["env_name"], wrappers=wrappers, wrapper_kwargs=wrapper_kwargs, ignore_keyword=ignore_keyword)
env = make_vec_env(make_env_fn, 
                    n_envs=config["environment"]["n_envs"], 
                    env_kwargs={"wrappers": config["environment"]["wrappers"], "wrapper_kwargs": config["environment"]["wrapper_kwargs"]},
                    monitor_kwargs={"allow_early_resets": True},
                    seed=config["seed"], vec_env_cls=DummyVecEnv)

pybullet build time: Sep 15 2023 11:13:12


argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


In [7]:
learner = config["learner_class"].load(os.path.join(folder_path, "evaluations", "best_model.zip"), env=env)

In [33]:
def record_video(video_output_path, env, learner, task=None, render_success=False, seed=None):
    if task is None:
        weights = np.zeros((1, learner.scheduler.reward_dim))
        weights[:, -1] = 1
    env.seed(seed)
    obs = env.reset()

    terminated = False
    render_imgs = []
    while not terminated:
        # obs = obs.reshape(1, -1)
        # print(obs)
        act = learner.predict(obs, weights=weights, deterministic=True)[0]
        obs, reward, terminated, info = env.step(act)
        
        img = env.render()
        if render_success:
            if reward > 0:
                # make a green border around the image
                img[:8, :, :] = [0, 255, 0]
                img[-8:, :, :] = [0, 255, 0]
                img[:, :8, :] = [0, 255, 0]
                img[:, -8:, :] = [0, 255, 0]
            else:
                # make a red border around the image
                img[:8, :, :] = [0, 0, 255]
                img[-8:, :, :] = [0, 0, 255]
                img[:, :8, :] = [0, 0, 255]
                img[:, -8:, :] = [0, 0, 255]
    
        render_imgs.append(img)
        
    print(reward)    
    height, width, _ = render_imgs[0].shape

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(video_output_path, fourcc=fourcc, fps=10, frameSize=(width, height))

    for frame in render_imgs:
        out.write(frame)
        
    out.release()
    
    os.system(f"ffmpeg -hide_banner -loglevel error -r 25 -i {video_output_path} -vcodec libx264 video.mp4")
    os.system(f"mv video.mp4 {video_output_path}")

In [9]:
video_path = os.path.join(folder_path, "video.mp4")
record_video(video_path, env, learner)
HTML(f'<video alt="test" controls><source src="{video_path}" type="video/mp4"></video>')

/home/benjamin/miniconda3/envs/rew_curr/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/home/benjamin/miniconda3/envs/rew_curr/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/home/benjamin/miniconda3/envs/rew_curr/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/home/benjamin/miniconda3/envs/rew_curr/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.

[0.]


# Render environment

In [ ]:
import panda_gym
import gymnasium as gym

env = gym.make("PandaMultiRewardStackDense-v3", renderer="OpenGL")
obs = env.reset()
env.render()

# Batch video generation

In [39]:
folder = "/home/benjamin/RewardCurriculum/results/panda_stack_long"
folders = [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isdir(os.path.join(folder, f)) and f != "Videos"]

generate_algorithms = ["settersolver", "alpgmm", "currot"]
runs_per_algorithm = 3

os.makedirs(os.path.join(folder, "Videos"), exist_ok=True)
for algorithm in generate_algorithms:
    os.makedirs(os.path.join(folder, "Videos", algorithm), exist_ok=True)
    
total_gen_videos = defaultdict(int)

env = None
seed = 0

for folder_path in tqdm.tqdm(folders):
    algorithm = folder_path.split("_")[-2]
    
    if algorithm in generate_algorithms:
        args = Namespace()
        args.env_name = json.load(open(os.path.join(folder_path, "config.json"), "r"))["environment"]["env_name"]
        args.seed = json.load(open(os.path.join(folder_path, "config.json"), "r"))["seed"]
        args.config_path = os.path.join(folder_path, "config_original.json")
        args.continue_from = None

        remaining_args = ["--environment.wrapper_kwargs.0.reward_threshold", "-0.05"]
        # remaining_args.__setattr__[] = "-0.05"
        config = get_config(args.config_path, args, remaining_args)
        config["environment"]["wrappers"] += ["SingleTaskRewardWrapper"]
        config["environment"]["wrapper_kwargs"] += [{}]
        
        if env is None:
            make_env_fn = lambda wrappers, wrapper_kwargs, ignore_keyword="ignore" : get_env(config["environment"]["env_name"], wrappers=wrappers, wrapper_kwargs=wrapper_kwargs, ignore_keyword=ignore_keyword)
            env = make_vec_env(make_env_fn, 
                        n_envs=config["environment"]["n_envs"], 
                        env_kwargs={"wrappers": config["environment"]["wrappers"], "wrapper_kwargs": config["environment"]["wrapper_kwargs"]},
                        monitor_kwargs={"allow_early_resets": True},
                        seed=config["seed"], vec_env_cls=DummyVecEnv)
        
        learner = config["learner_class"].load(os.path.join(folder_path, "evaluations", "best_model.zip"), env=env)
        
        for _ in range(runs_per_algorithm):
            video_path = os.path.join(folder, "Videos", algorithm, f"video_{total_gen_videos[algorithm]}.mp4")
            record_video(video_path, env, learner, render_success=True, seed=seed)
            total_gen_videos[algorithm] += 1
            seed += 1
        

  0%|          | 0/61 [00:00<?, ?it/s]

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
[0.]
[0.]


  2%|▏         | 1/61 [00:32<32:45, 32.76s/it]

[0.]
[0.]
[0.]


  3%|▎         | 2/61 [01:05<32:06, 32.65s/it]

...

[0.]
[0.]
[0.]


 89%|████████▊ | 54/61 [15:48<02:13, 19.14s/it]

[0.]
[0.]
[0.]


100%|██████████| 61/61 [16:21<00:00, 16.09s/it]

[0.]
